In [2]:
import json
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

In [4]:
with open('../input/metrics_dataset-traffic-test.json', 'r') as f:
    training_data_a = json.load(f)

with open('../input/metrics_dataset-domesticDeclarations.json', 'r') as f:
    training_data_b = json.load(f)

with open('../input/metrics_dataset.json', 'r') as f:
    training_data_c = json.load(f)

In [5]:
data = training_data_a['metrics']
data.extend(training_data_b['metrics'])
data.extend(training_data_c['metrics'])

### Flan Experiment

In [6]:
examples = "-time: average time from opened to closed, Elapsed time between an action requirement is notified until the action is performed\n-count: number of activities reopened, number of users within the black list\n-data: Average of declared costs detailed per trip, the value of state"

In [7]:
real_labels = []
predicted_labels = []

counter = 0
for phrase in data:
    if phrase['type'] not in ['time', 'count', 'data']:
        continue
    counter += 1
    print(f"Predicting {counter}/{len(data)}")
    real_labels.append(phrase['type'])

    prompt = f"Given this sentence:\n{phrase['description']}\nChoose the correct label:\n-time\n-count\n-data\n\nEXAMPLES\n{examples}"
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    outputs = model.generate(input_ids)
    predicted_labels.append(tokenizer.decode(outputs[0], skip_special_tokens=True))    

Predicting 1/81


/home/ppibot/anaconda3/envs/myenv/lib/python3.9/site-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Predicting 2/81
Predicting 3/81
Predicting 4/81
Predicting 5/81
Predicting 6/81
Predicting 7/81
Predicting 8/81
Predicting 9/81
Predicting 10/81
Predicting 11/81
Predicting 12/81
Predicting 13/81
Predicting 14/81
Predicting 15/81
Predicting 16/81
Predicting 17/81
Predicting 18/81
Predicting 19/81
Predicting 20/81
Predicting 21/81
Predicting 22/81
Predicting 23/81
Predicting 24/81
Predicting 25/81
Predicting 26/81
Predicting 27/81
Predicting 28/81
Predicting 29/81
Predicting 30/81
Predicting 31/81
Predicting 32/81
Predicting 33/81
Predicting 34/81
Predicting 35/81
Predicting 36/81
Predicting 37/81
Predicting 38/81
Predicting 39/81
Predicting 40/81
Predicting 41/81
Predicting 42/81
Predicting 43/81
Predicting 44/81
Predicting 45/81
Predicting 46/81
Predicting 47/81
Predicting 48/81
Predicting 49/81
Predicting 50/81
Predicting 51/81
Predicting 52/81
Predicting 53/81
Predicting 54/81
Predicting 55/81
Predicting 56/81
Predicting 57/81
Predicting 58/81
Predicting 59/81
Predicting 60/81
Predi

In [8]:
precision, recall, f1, _ = precision_recall_fscore_support(real_labels, predicted_labels, average='macro')
acc = accuracy_score(real_labels, predicted_labels)

print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")

Accuracy: 0.9487179487179487
Precision: 0.9648879648879648
Recall: 0.8162393162393163
F1: 0.8625356125356126


In [9]:
report = classification_report(real_labels, predicted_labels)
print(report)

              precision    recall  f1-score   support

       count       0.95      0.95      0.95        39
        data       1.00      0.50      0.67         4
        time       0.95      1.00      0.97        35

    accuracy                           0.95        78
   macro avg       0.96      0.82      0.86        78
weighted avg       0.95      0.95      0.94        78



### Old Classifier

In [10]:
model = AutoModelForSequenceClassification.from_pretrained("../ppinat/models/TextClassification")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

real_labels = []
predicted_labels = []

counter = 0
for phrase in data:
    if phrase['type'] not in ['time', 'count', 'data']:
        continue
    counter += 1
    print(f"Predicting {counter}/{len(data)}")
    real_labels.append(phrase['type'])

    words = phrase['description'].split()
    tokens  = tokenizer(words, return_tensors='pt', truncation=True, is_split_into_words=True)

    metric_type = model(**tokens)["logits"].argmax(-1).tolist()[0]
    if metric_type == 0:
        type = "time"
    elif metric_type == 1:
        type = "count"
    else:
        type = "data"

    predicted_labels.append(type)

precision, recall, f1, _ = precision_recall_fscore_support(real_labels, predicted_labels, average='macro')
acc = accuracy_score(real_labels, predicted_labels)

print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {f1}")


Predicting 1/81
Predicting 2/81
Predicting 3/81
Predicting 4/81
Predicting 5/81
Predicting 6/81
Predicting 7/81
Predicting 8/81
Predicting 9/81
Predicting 10/81
Predicting 11/81
Predicting 12/81
Predicting 13/81
Predicting 14/81
Predicting 15/81
Predicting 16/81
Predicting 17/81
Predicting 18/81
Predicting 19/81
Predicting 20/81
Predicting 21/81
Predicting 22/81
Predicting 23/81
Predicting 24/81
Predicting 25/81
Predicting 26/81
Predicting 27/81
Predicting 28/81
Predicting 29/81
Predicting 30/81
Predicting 31/81
Predicting 32/81
Predicting 33/81
Predicting 34/81
Predicting 35/81
Predicting 36/81
Predicting 37/81
Predicting 38/81
Predicting 39/81
Predicting 40/81
Predicting 41/81
Predicting 42/81
Predicting 43/81
Predicting 44/81
Predicting 45/81
Predicting 46/81
Predicting 47/81
Predicting 48/81
Predicting 49/81
Predicting 50/81
Predicting 51/81
Predicting 52/81
Predicting 53/81
Predicting 54/81
Predicting 55/81
Predicting 56/81
Predicting 57/81
Predicting 58/81
Predicting 59/81
Predic

In [11]:
report = classification_report(real_labels, predicted_labels)
print(report)

              precision    recall  f1-score   support

       count       0.97      0.92      0.95        39
        data       0.60      0.75      0.67         4
        time       0.97      1.00      0.99        35

    accuracy                           0.95        78
   macro avg       0.85      0.89      0.87        78
weighted avg       0.95      0.95      0.95        78

